# Margins of error in the ACS

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mmann1123/pytidycensus/blob/main/examples/03_margins_of_error.ipynb)

Understanding and working with uncertainty in American Community Survey data.

In [ ]:
import pytidycensus as tc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

tc.set_census_api_key("YOUR API KEY GOES HERE")

## Understanding ACS Uncertainty

Unlike decennial Census counts, ACS data are estimates with margins of error.

In [ ]:
# Example: Aging populations in Ramsey County, MN
age_vars = [f"B01001_0{i:02d}" for i in range(20, 26)] + [f"B01001_0{i:02d}" for i in range(44, 50)]

ramsey = tc.get_acs(
    geography="tract",
    variables=age_vars,
    state="MN", 
    county="Ramsey",
    year=2022
)

# Show cases where MOE exceeds estimate
ramsey['moe_ratio'] = ramsey['B01001_020_moe'] / ramsey['value']  # Example MOE column
print("Cases where margin of error exceeds estimate:")
print(ramsey[ramsey['moe_ratio'] > 1].head())

## Aggregating Data and MOE Calculations

When combining estimates, we need to properly calculate the margin of error.

In [ ]:
# Custom MOE calculation functions (simplified versions)
def moe_sum(moes, estimates):
    """Calculate MOE for sum of estimates"""
    return np.sqrt(sum(moe**2 for moe in moes))

# Aggregate population over 65 by tract
ramsey_65plus = ramsey.groupby('GEOID').agg({
    'value': 'sum',
    'B01001_020_moe': lambda x: moe_sum(x, ramsey.loc[x.index, 'value'])
}).rename(columns={'B01001_020_moe': 'moe_sum'})

print("Aggregated estimates with proper MOE calculation:")
print(ramsey_65plus.head())

## Visualization with Confidence Intervals

In [ ]:
# Create error bar plot showing uncertainty
fig, ax = plt.subplots(figsize=(12, 8))

sample_data = ramsey_65plus.head(10)
x = range(len(sample_data))

ax.errorbar(x, sample_data['value'], yerr=sample_data['moe_sum'], 
           fmt='o', capsize=5, capthick=2)
ax.set_xlabel('Census Tract')
ax.set_ylabel('Population 65+')
ax.set_title('Population 65+ by Census Tract with Margins of Error')
plt.xticks(x, [f'Tract {i+1}' for i in x], rotation=45)
plt.tight_layout()
plt.show()